# Assignment 5

## Question1 - Use boosting with your final project dataset:
Choose any type of boosting: XGBoost, LightGBM or CatBoost and train it with your final project dataset and predict whether a pet is adopted within the 30 days stay time in the animal center. Don't forget your test set! It is test_features.csv file. Test labels are in y_test.csv file.

Let's look at the fields in this dataset:
* Pet ID - Unique ID of pet
* Outcome Type - State of pet at the time of recording the outcome
* Sex upon Outcome - Sex of pet at outcome
* Name - Name of pet 
* Found Location - Found location of pet before entered the center
* Intake Type - Circumstances bringing the pet to the center
* Intake Condition - Health condition of pet when entered the center
* Pet Type - Type of pet
* Sex upon Intake - Sex of pet when entered the center
* Breed - Breed of pet 
* Color - Color of pet 
* Age upon Intake Days - Age of pet when entered the center (days)
* Time at Center - Time at center (0 = less than 30 days; 1 = more than 30 days). This is the value to predict.

Let's read our dataset using the Pandas library. This will load the data into a data frame, it acts like a data table.

In [11]:
import pandas as pd

data = pd.read_csv("training.csv")

In [14]:
# Display first 10 rows
data.head(10)
#data.shape


,Pet ID,Outcome Type,Sex upon Outcome,Name,Found Location,Intake Type,Intake Condition,Pet Type,Sex upon Intake,Breed,Color,Age upon Intake Days,Time at Center
19225,A789010,Transfer,Intact Male,*Carter,Taylor Lane in Manor (TX),Stray,Normal,Dog,Intact Male,Cardigan Welsh Corgi Mix,Tan,730,0
48079,A784744,Adoption,Spayed Female,NaN,Lakeline Mall Drive in Austin (TX),Stray,Normal,Dog,Intact Female,Australian Cattle Dog Mix,Blue Tick/Red,60,0
12962,A732495,Adoption,Spayed Female,*Mistyfrnkln,Austin (TX),Owner Surrender,Normal,Cat,Intact Female,Domestic Medium Hair Mix,Blue Tabby/White,60,0
25105,A677916,Return to Owner,Spayed Female,Pixy,Austin (TX),Owner Surrender,Normal,Cat,Spayed Female,Domestic Shorthair Mix,Brown Tabby/White,365,0
34414,A797275,Rto-Adopt,Neutered Male,Rocket,605 Masterson Pass in Austin (TX),Stray,Normal,Dog,Intact Male,Yorkshire Terrier,Brown,365,0
41597,A731300,Adoption,Neutered Male,*Rodger,Austin (TX),Owner Surrender,Normal,Cat,Intact Male,Siamese Mix,Lilac Point,60,0
65810,A796397,Transfer,Neutered Male,*Eduardo,7002 Hillcroft in Austin (TX),Stray,Normal,Cat,Intact Male,Domestic Shorthair Mix,Orange Tabby/White,30,1
28019,A752207,Transfer,Neutered Male,NaN,Austin (TX),Stray,Normal,Dog,Intact Male,Miniature Schnauzer Mix,Gray/White,365,0
14824,A812595,Adoption,Spayed Female,Gnocchi,5561 Tierra Alta Circle in Travis (TX),Stray,Normal,Cat,Intact Female,Domestic Shorthair,Blue Tabby/White,30,0
36683,A689449,Adoption,Spayed Female,*Bizzy,9520 Quail Ct in Austin (TX),Stray,Normal,Cat,Intact Female,Domestic Shorthair Mix,Black,21,1


In [5]:
# Test data
test_df = pd.read_csv("test_features.csv")

test_df.head()
test_df.shape

(23846, 12)

In [13]:
from sklearn.model_selection import train_test_split
data = data.sample(frac = 0.1, random_state=10)

In [28]:
data=data[['Pet ID','Outcome Type','Sex upon Outcome','Name','Found Location','Intake Type','Intake Condition','Pet Type','Sex upon Intake',
      'Breed','Color','Age upon Intake Days','Time at Center']]
data.dropna(inplace=True)
#if data['Time at Center']==0:
 #   (data['Age upon Intake Days']<30)
#if data['Time at Center']==1:
#    (data['Age upon Intake Days']>30)
data = data[(data['Age upon Intake Days']>30) | (data['Age upon Intake Days']<30)]
    
cols = ['Pet ID','Outcome Type','Sex upon Outcome','Name','Found Location','Intake Type','Intake Condition','Pet Type','Sex upon Intake',
      'Breed','Color']
for item in cols:
    data[item] = data[item].astype("category").cat.codes +1
    
train, test, y_train, y_test = train_test_split(data.drop(['Time at Center'], axis=1), data['Time at Center'], random_state=10, test_size=0.25)

In [31]:
import xgboost as xgb
from sklearn import metrics

def auc(m,train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

# Parameter Tuning
model = xgb.XGBClassifier()

model = xgb.XGBClassifier(max_depth=50, min_child_weight=1,  n_estimators=200, n_jobs=-1 ,learning_rate=0.16,eval_metric='mlogloss',use_label_encoder=False)
model.fit(train,y_train)

auc(model, train, test)

(1.0, 0.8806014619418402)